In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [2]:
import psycopg2
import pandas as pd
import numpy as np
import config.config as conf
import pickle
import lib.preprocess.preprocess as pp
import lib.preprocess.SectionExtractor as se

In [3]:
htmlp = pp.HTMLParser()
codep = pp.CodeSectionParser()
ts = se.SectionExtractor()

In [4]:
df = pd.read_csv('../../data/241119_annotation_50_for_chatgpt.csv')

In [5]:
df.columns

Index(['Unnamed: 0', 'id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set',
       'A_Difficulty_class', 'A_General_rule_set', 'A_Granular_breakdown'],
      dtype='object')

In [6]:
# 'id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set',
#        'Q_Granular_breakdown'

In [7]:
q_df = df[['id', 'link', 'Q_Difficulty_class', 'Q_General_rule_set']]
a_df = df[['id', 'link', 'A_Difficulty_class', 'A_General_rule_set','A_Granular_breakdown']]

In [8]:
q_df = q_df[~(q_df['Q_Difficulty_class'] =='-')]
a_df = a_df[~(a_df['A_Difficulty_class'] =='-')]

In [9]:
q_df.head()

,id,link,Q_Difficulty_class,Q_General_rule_set
0,77422970,https://stackoverflow.com/questions/77422970,Advanced,A3 - Data & AI
1,77155599,https://stackoverflow.com/questions/77155599,Intermediate,I3 - Frameworks
2,77291783,https://stackoverflow.com/questions/77291783,Intermediate,I3 - Frameworks
3,77150535,https://stackoverflow.com/questions/77150535,Advanced,A3 - Data & AI
5,77068633,https://stackoverflow.com/questions/77068633,Basic,B1 - Syntax


In [10]:
q_df

,id,link,Q_Difficulty_class,Q_General_rule_set
0,77422970,https://stackoverflow.com/questions/77422970,Advanced,A3 - Data & AI
1,77155599,https://stackoverflow.com/questions/77155599,Intermediate,I3 - Frameworks
2,77291783,https://stackoverflow.com/questions/77291783,Intermediate,I3 - Frameworks
3,77150535,https://stackoverflow.com/questions/77150535,Advanced,A3 - Data & AI
5,77068633,https://stackoverflow.com/questions/77068633,Basic,B1 - Syntax
6,77020489,https://stackoverflow.com/questions/77020489,Intermediate,I1 - Advanced Features
8,76917593,https://stackoverflow.com/questions/76917593,Basic,B1 - Syntax
10,76798081,https://stackoverflow.com/questions/76798081,Basic,B1 - Syntax
11,76774917,https://stackoverflow.com/questions/76774917,Basic,B2 - Environment
12,76675433,https://stackoverflow.com/questions/76675433,Intermediate,I2 - Performance


In [11]:
q_id_str = str([x for x in q_df['id']])[1:-1]
a_id_str = str([x for x in a_df['id']])[1:-1]

In [12]:
q_id_str

'77422970, 77155599, 77291783, 77150535, 77068633, 77020489, 76917593, 76798081, 76774917, 76675433, 76427264, 76331128, 76585733, 76367594, 76568510, 76481433, 76115814, 76059210, 76040179, 75870881, 75995957, 75965577, 75711831, 75941121, 75789316, 75942768, 75928730, 75492157, 75352739, 75333570, 75524574, 75305824, 75266730, 75259252, 75243277, 75022065, 74936129, 75099305, 74793352, 74727945, 74627211, 74700764, 77538219, 77464645'

In [13]:
q_postypeid="'1'"
a_postypeid="'2'"



In [14]:
# q_sql = """select a.id 
#                 , 'Here is the question about software development. The title of the question is "'||a.title||'". and body of the question is "'||b.body||'". Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'
#             from posts a 
#                , postsbody b
#             where a.id = b.id
#               and a.creationdate between '2021-11-30' and '2023-11-30' 
#               and a.posttypeid = {0} 
#               and a.tags like '%<python>%' 
#               and a.id in ({1})
# """

In [15]:
q_sql = """select a.id 
                , a.title
                , b.body
            from posts a 
               , postsbody b
            where a.id = b.id
              and a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [16]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [17]:
q_output = pd.DataFrame(rows, columns = [
  'id'
  ,'title'
  , 'body'
])


In [18]:
q_output.head()

,id,title,body
0,74627211,"Why is this gunicorn, flask, bokeh application...","<p>I am learning to use gunicorn, flask and bo..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...
3,74793352,matplotlib pyplot display ticks and values whi...,<p>I have a plot which looks like this:</p>\n<...
4,74936129,Least Squares Fitting/Power Fit - Why is my va...,<p>so I have to find the coefficients of this ...


In [19]:
q_output

,id,title,body
0,74627211,"Why is this gunicorn, flask, bokeh application...","<p>I am learning to use gunicorn, flask and bo..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...
3,74793352,matplotlib pyplot display ticks and values whi...,<p>I have a plot which looks like this:</p>\n<...
4,74936129,Least Squares Fitting/Power Fit - Why is my va...,<p>so I have to find the coefficients of this ...
5,75022065,pandas transform the dataframe by adding custo...,<p>I am very new to pandas and am trying to fi...
6,75099305,Strange error: TypeError: forward() takes 2 po...,<p>I am trying to train a VAE model. However I...
7,75243277,Read a file and generate a list of lists using...,<p>I'd like to read a file with the following ...
8,75259252,Resize bin size,<p>I need to change the bin size of the Y-axis...
9,75266730,How to compare two dataframes in pandas withou...,<p>I want to compare two dataframes and find p...


In [20]:
q_output['c_body'] = q_output['body'].apply(lambda x : codep(x))

post_str : <p>I am learning to use gunicorn, flask and bokeh.
I am trying to set up a little example to use a bokeh app with multiuser. I am working on a debian system on an AWS server with a static IP. I used the <a href="https://github.com/bokeh/bokeh/blob/2.4.3/examples/howto/server_embed/flask_gunicorn_embed.py" rel="nofollow noreferrer">example from git</a> and edited it in the way I think it should work. I got it to run with this command:</p>
<pre><code>gunicorn3 -w 1 --bind 0.0.0.0:8000 wsgi:app
</code></pre>
<p>But when I change the workers to 4 and the binded port from 5006 to 0 (flaskapp.py), I get this error in the browser console:</p>
<blockquote>
<p>:34163/bkapp/autoload.js?bokeh-autoload-element=4755&amp;bokeh-app-path=/bkapp&amp;bokeh-absolute-url=http://35.XXX.XXX.153:34163/bkapp:1          Failed to load resource: net::ERR_CONNECTION_TIMED_OUT</p>
</blockquote>
<p>Can someone tell me, why I get this error?</p>
<p>Here is the relevant code:</p>
<p><strong>flaskapp.py</s

In [21]:
q_output['t_body'] = q_output[['c_body', 'body']].apply(lambda row: ts.get_text_section({'body': row['body'], 'code_sections': row['c_body']['code_sections']}), axis=1)

In [22]:
q_output['t_body']

0     <p>I am learning to use gunicorn, flask and bo...
1     <p>I want to replace multiple strings in a lis...
2     <p>I have this function that I want to test:</...
3     <p>I have a plot which looks like this:</p>\n<...
4     <p>so I have to find the coefficients of this ...
5     <p>I am very new to pandas and am trying to fi...
6     <p>I am trying to train a VAE model. However I...
7     <p>I'd like to read a file with the following ...
8     <p>I need to change the bin size of the Y-axis...
9     <p>I want to compare two dataframes and find p...
10    <p>I have a workflow using an R script and Pyt...
11    <p>im trying to create a data set with a uniqu...
12    <ol>\n<li>Given a dataset</li>\n</ol>\n<pre><c...
13    <p>model.fit(X_train, y_train, batch_size=128,...
14    <p>I am using django-rest-knox 4.1.0 . In its ...
15    <p>Some times, when we call an .exe file in co...
16    <p>suppose I have a list</p>\n<pre><code>\n\n\...
17    <p>I have a dataframe <code>df</code> with

In [23]:
q_output['clean_body'] = q_output['t_body'].apply(lambda x : htmlp.get_html_cleaned_str(x))

In [24]:
q_output.head()

,id,title,body,c_body,t_body,clean_body
0,74627211,"Why is this gunicorn, flask, bokeh application...","<p>I am learning to use gunicorn, flask and bo...","{'code_sections': [{'off_beg': 453, 'off_end':...","<p>I am learning to use gunicorn, flask and bo...","I am learning to use gunicorn, flask and bokeh..."
1,74700764,replace multiple strings in a list with a sing...,<p>I want to replace multiple strings in a lis...,"{'code_sections': [{'off_beg': 386, 'off_end':...",<p>I want to replace multiple strings in a lis...,I want to replace multiple strings in a list w...
2,74727945,pandas unit test AssertionError: DataFrame.ind...,<p>I have this function that I want to test:</...,"{'code_sections': [{'off_beg': 60, 'off_end': ...",<p>I have this function that I want to test:</...,I have this function that I want to test:\n\n\...
3,74793352,matplotlib pyplot display ticks and values whi...,<p>I have a plot which looks like this:</p>\n<...,"{'code_sections': [{'off_beg': 328, 'off_end':...",<p>I have a plot which looks like this:</p>\n<...,I have a plot which looks like this:\n\nThe va...
4,74936129,Least Squares Fitting/Power Fit - Why is my va...,<p>so I have to find the coefficients of this ...,"{'code_sections': [{'off_beg': 435, 'off_end':...",<p>so I have to find the coefficients of this ...,so I have to find the coefficients of this fun...


In [25]:
q_output = pd.merge(q_output, q_df[['id', 'Q_Difficulty_class', 'Q_General_rule_set']], on = ['id'] )

In [26]:
q_output_link_ver = q_output.copy()

In [27]:
q_output['answer'] = 'Difficulty class : '+q_output['Q_Difficulty_class'].map(str)+" | "+ 'General rule set : '+q_output['Q_General_rule_set'].map(str)

In [28]:
q_output['question'] = 'Here is the question about software development. The title of the question is "'+q_output['title'].map(str)+'". and body of the question is "'+q_output['clean_body'].map(str)+'". Please answer briefly about the "difficulty class" and "general ruleset" for the question'


In [29]:
q_output[['question', 'answer']]

,question,answer
0,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
1,Here is the question about software developmen...,Difficulty class : Basic | General rule set : ...
2,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
3,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
4,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
5,Here is the question about software developmen...,Difficulty class : Basic | General rule set : ...
6,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
7,Here is the question about software developmen...,Difficulty class : Basic | General rule set : ...
8,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...
9,Here is the question about software developmen...,Difficulty class : Intermediate | General rule...


In [30]:
q_output[['id', 'question', 'answer']].to_csv('../../data/q_output.csv')

In [31]:
# [34,  3, 37, 33]
q_output[['body', 'clean_body']].to_csv('chk.csv')

In [32]:
# q_output.iloc[[14, 25, 13, 21], :].to_csv('chk.csv')

In [33]:
q_sql_v1 = """select id, 'https://stackoverflow.com/questions/'||a.id as link
            from posts a 
            where a.creationdate between '2021-11-30' and '2023-11-30' 
              and a.posttypeid = {0} 
              and a.tags like '%<python>%' 
              and a.id in ({1})
"""

In [34]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute(q_sql_v1.format(q_postypeid, q_id_str))
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [35]:
q_output_link = pd.DataFrame(rows, columns = [
  'id', 'link'
  
])


In [36]:
q_output_link.head()

,id,link
0,74627211,https://stackoverflow.com/questions/74627211
1,74700764,https://stackoverflow.com/questions/74700764
2,74727945,https://stackoverflow.com/questions/74727945
3,74793352,https://stackoverflow.com/questions/74793352
4,74936129,https://stackoverflow.com/questions/74936129


In [37]:
q_output_link = pd.merge(q_output_link, q_df[['id', 'Q_Difficulty_class', 'Q_General_rule_set', 'Q_Granular_breakdown']], on = ['id'] )

KeyError: "['Q_Granular_breakdown'] not in index"

In [ ]:
q_output_link

In [54]:
q_output_link['answer'] = 'Difficulty class : '+q_output_link['Q_Difficulty_class'].map(str)+" | "+ 'General rule set : '+q_output_link['Q_General_rule_set'].map(str)+" | "+ 'Granular breakdown : '+q_output_link['Q_Granular_breakdown'].map(str) 

In [56]:
q_output_link['question'] = 'Here is the question about software development. Please access to the link here. "'+q_output_link['link'].map(str)+'". and check the title and body of the question. Please answer briefly about the "difficulty class" and "general ruleset" and "grenular breakdown" for the question'


In [59]:
q_output_link[['id','question', 'answer']].to_csv('../../data/q_output_link.csv')

In [ ]:
q_output_link.head()